In [1]:
import matlab.engine
from delfi.simulator.BaseSimulator import BaseSimulator
import scipy.io
import os

import delfi.distribution as dd
import delfi.generator as dg
import numpy as np
import delfi.inference as infer

In [2]:
def ConfirmationBiasSimulator(params, signals, engine):
    # run matlab code run.vectorized and read results
    print("Hello I am in Simulator")
    print(type(signals))
    print(type(params[0]))
    p = {'prior_C': float(params[0]),
      'gamma':   float(params[1]),
      'samples': float(params[2]),
      'lapse':   float(params[3]),
      'sensor_noise': float(params[4])}
    signals = np.ndarray.tolist(signals)
    print(type(p))
    print(p)
    sim_results = engine.Model.simple_script(p, signals)
    return sim_results
# define a simulator class linking to the real simulator

class ConfirmationBias(BaseSimulator):
    
    def __init__(self, signal, engine, seed = None):
        
        dim_param = 5
        super().__init__(dim_param = dim_param,seed = seed)
        self.signal = signal
        self.simulate = ConfirmationBiasSimulator
        self.engine = engine
        
    def gen_single(self,params):
        
        sim_results = self.simulate(params,self.signal, self.engine)
        return sim_results

In [3]:
from delfi.summarystats.BaseSummaryStats import BaseSummaryStats
class ConfirmationBiasStats(BaseSummaryStats):
    def __init__(self,choice):
        self.choice = choice
    def calc(self):
        stats = self.choice
        return stats

In [4]:
# load the MATLAB engine
engine = matlab.engine.start_matlab()

datafolder = '../dscData'
filename = 'syntheticData_priorC5.mat'
syntheticData = scipy.io.loadmat(os.path.join(datafolder, filename))
# load data
choice = syntheticData['sim_results'][0][0][1]
signals = syntheticData['signals']

# define prior over model parameters
seed_p = 2
prior_min = np.array([0,0,1,0,0])
prior_max = np.array([1,1,100,1,5])
prior =  dd.Uniform(lower = prior_min , upper = prior_max,seed = seed_p)

# inference parameters
seed_inf = 1

pilot_samples = 2000

# training schedule
n_train = 2000
n_rounds = 1

# fitting setup
minibatch = 100
epochs = 100
val_frac = 0.05

# network setup
n_hiddens = [50,50]

# convenience
prior_norm = True

# MAF parameters
density = 'maf'
n_mades = 5         # number of MADES


# define generator class
s = ConfirmationBiasStats(choice)
# use multiple processes of simulators
n_processes = 1
seeds_m = np.arange(1,n_processes+1,1)
m = []
for i in range(n_processes):
    m.append(ConfirmationBias(signals, engine, seeds_m[i]))
g = dg.MPGenerator(models=m, prior=prior, summary=s)

In [ ]:
# do inference
# inference object
res = infer.SNPEC(g,
                obs=choice,
                n_hiddens=n_hiddens,
                seed=seed_inf,
                pilot_samples=pilot_samples,
                n_mades=n_mades,
                prior_norm=prior_norm,
                density=density)

# train
log, _, posterior = res.run(
                    n_train=n_train,
                    n_rounds=n_rounds,
                    minibatch=minibatch,
                epochs=epochs,
                silent_fail=False,
                proposal='prior',
                val_frac=val_frac,
                verbose=True,)

Hello I am in Simulator
<class 'numpy.ndarray'>
<class 'numpy.float64'>
<class 'dict'>
{'prior_C': 0.43599490214200376, 'gamma': 0.025926231827891333, 'samples': 55.41658530999221, 'lapse': 0.4353223926182769, 'sensor_noise': 2.101839010437445}


In [5]:
import matlab.engine
import time
eng = matlab.engine.start_matlab()

a = 2
b = 2
t = time.time()
a = eng.Model.simple_script(a, b, nargout=1)
elapsed = time.time() - t

print(a)
eng.quit()

4


In [ ]:
type(signals)

In [ ]:
params

In [ ]:
par